In [ ]:
from autumn.tools.project import get_project
from matplotlib import pyplot as plt
from autumn.tools.plots.utils import REF_DATE
import pandas as pd

In [ ]:
project = get_project("covid_19", "manila")

In [ ]:
# custom_params = project.param_set.baseline.update(dict(contact_rate = 0.031))
model = project.run_baseline_model(project.param_set.baseline)
derived_df = model.get_derived_outputs_df()

In [ ]:
outputs = ["new_hospital_admissions", "tts_casesXagegroup_5"]

In [ ]:
for output in outputs:
    fig = plt.figure(figsize=(12, 8))
    plt.style.use("ggplot")
    axis = fig.add_subplot()
    axis = derived_df[output].plot()
    axis.set_ylabel(output)

# Cumulative TTS calculations

In [ ]:
print(derived_df.index > pd.Timestamp("2021-01-01"))